In [1]:
import sys

In [6]:
# global config
DATA_PATH = '/data/data/'
MODEL_PATH = '/data/models/'
DATABASE_PATH = '/data/database/'
PREFIX='.npy'

In [7]:
# Targic data
TARGET_PATH = 'T1019s2/'
TARGET_NAME = 'T1019s2'

In [8]:
# Set up target fiel
TARGET_DIR = DATA_PATH + TARGET_PATH
TARGET = TARGET_DIR + TARGET_NAME
TARGET_SEQ=TARGET+'.seq'
print(TARGET_SEQ)

/data/data/T1019s2/T1019s2.seq


In [9]:
%%bash
export PATH="/hh-suite/build/bin:/hh-suite/build/scripts:$PATH"

TARGET="T1019s2"
TARGET_DIR="/data/data/T1019s2"
TARGET_SEQ="${TARGET_DIR}/${TARGET}.seq" # fasta format
PLMDCA_DIR="/plmDCA/plmDCA_asymmetric_v2/"

DATABASE_DIR="/data/database/hhblits/pdb70"
FEATURE_GENERATOR="/alphafold_pytorch/feature.py"

# generate domain crops from target seq
python3 $FEATURE_GENERATOR -s $TARGET_SEQ -c

for domain in ${TARGET_DIR}/*.seq; do
        out=${domain%.seq}
        echo "Generate MSA files for ${out}"
        hhblits -cpu 4 -i ${out}.seq -d ${DATABASE_DIR} -oa3m ${out}.a3m -ohhm ${out}.hhm -n 3
        reformat.pl ${out}.a3m ${out}.fas
        psiblast -subject ${out}.seq -in_msa ${out}.fas -out_ascii_pssm ${out}.pssm
        python3 $FEATURE_GENERATOR -s ${out}.seq -f
done

# make target features data and generate ungap target aln file for plmDCA
python3 $FEATURE_GENERATOR -s $TARGET_SEQ -f

cd $PLMDCA_DIR
for aln in ${TARGET_DIR}/*.aln; do
        echo "calculate plmDCA for $aln"
        octave plmDCA.m $aln
done

cd $TARGET_DIR

# run again to update target features data
python3 $FEATURE_GENERATOR -s $TARGET_SEQ -f

Generate MSA files for /data/data/T1019s2/T1019s2-l64_s0
Query         T1019s2-l64_s0
Match_columns 64
No_of_seqs    1 out of 1
Neff          1
Searched_HMMs 100
Date          Sat Jan 30 23:02:59 2021
Command       hhblits -cpu 4 -i /data/data/T1019s2/T1019s2-l64_s0.seq -d /data/database/hhblits/pdb70 -oa3m /data/data/T1019s2/T1019s2-l64_s0.a3m -ohhm /data/data/T1019s2/T1019s2-l64_s0.hhm -n 3 

 No Hit                             Prob E-value P-value  Score    SS Cols Query HMM  Template HMM
  1 2DK7_A Transcription elongatio  27.8     5.3 0.00064   19.3   0.0   14   49-62     28-41  (73)
  2 1IYB_A ribonuclease M5(E.C.3.1  25.2     5.5 0.00076   21.2   0.0   27   11-37     97-123 (208)
  3 1UCD_A Ribonuclease MC(E.C.3.1  24.2     5.8 0.00082   20.6   0.0   27   11-37     84-110 (190)
  4 1W2Y_A DEOXYURIDINE 5'-TRIPHOS  21.1     8.2   0.001   21.9   0.0   31    8-38     55-85  (229)
  5 6YWX_a 60S ribosomal protein L  21.0      10   0.001   23.3   0.0   10   50-59     65-74  (225)
  6 

- 23:02:59.235 INFO: Search results will be written to /data/data/T1019s2/T1019s2-l64_s0.hhr

- 23:02:59.266 INFO: Searching 85757 column state sequences.

- 23:02:59.318 INFO: /data/data/T1019s2/T1019s2-l64_s0.seq is in A2M, A3M or FASTA format

- 23:02:59.318 INFO: Iteration 1

- 23:02:59.344 INFO: Prefiltering database

- 23:02:59.390 INFO: HMMs passed 1st prefilter (gapless profile-profile alignment)  : 298

- 23:02:59.391 INFO: HMMs passed 2nd prefilter (gapped profile-profile alignment)   : 100

- 23:02:59.391 INFO: HMMs passed 2nd prefilter and not found in previous iterations : 100

- 23:02:59.391 INFO: Scoring 100 HMMs using HMM-HMM Viterbi alignment

- 23:02:59.575 INFO: Alternative alignment: 0

- 23:02:59.587 INFO: 100 alignments done

- 23:02:59.587 INFO: Stop after DB-HHM: 100 because early stop  0.996413 < filter cutoff 1

- 23:02:59.587 INFO: Alternative alignment: 1

- 23:02:59.593 INFO: 9 alignments done

- 23:02:59.593 INFO: Alternative alignment: 2

- 23:02:59.593 I

In [10]:
import torch
import time
import argparse

import numpy as np
import multiprocessing as mp

sys.path.append('../alphafold_pytorch')
import utils
from alphafold import run_eval, ensemble
from pathlib import Path
from network import ContactsNet
from dataset import ProteinDataLoader
from datetime import datetime

In [11]:
targetFile=TARGET + PREFIX
outputDir= TARGET + "_out"
print(targetFile)
print(outputDir)
timestr = datetime.now().strftime('%Y_%m_%d_%H_%M_%S')

/data/data/T1019s2/T1019s2.npy
/data/data/T1019s2/T1019s2_out


In [12]:
print("Saving output to " + outputDir)

Saving output to /data/data/T1019s2/T1019s2_out


In [13]:
startTime = time.time()

# multi processer
ctx = mp.get_context('spawn')
numParallelProcesser = 0
processes = []

# multi gpus
isEvenlyDistributed = True
isGPU = True

devices = []
if torch.cuda.is_available() and isGPU :
    numDevices = torch.cuda.device_count()
    for deviceIndex in range(0, numDevices):
        devices.append("cuda:" + str(deviceIndex))
else:
    devices = ["cpu"]

target = targetFile.split('/')[-1].split('.')[0]

In [ ]:
# running loop
numDevices = len(devices)
count = 0

for replica in range(0, 4):
    mode = ["D","B"]
    if replica == 0 :
        mode = ["D","B", "T"]
        
    for m in mode:
        device = devices[count % numDevices]
        if isEvenlyDistributed:
            count = count + 1
        print("Lauching model: " + m + " " + str(replica) + " on device " + device)
        
        if m == 'D':
            modelType = "Distogram"
            modelPath = Path(MODEL_PATH + "/873731")
        elif m == 'B':
            modelType = "Background"
            modelPath = Path(MODEL_PATH + "/916425")
        elif m == 'T':
            modelType = "Torsion"
            modelPath = Path(MODEL_PATH + "/941521")
            
        outDir = outputDir + "/" + modelType + "/" + str(replica)
        Path(outDir).mkdir(parents=True, exist_ok=True)
        
        print("Input file: " + targetFile)
        print("Output dir: " + outDir)
        print(modelType + " model: " + str(modelPath))
        print("Replica: " + str(replica))
        print("Device: " + str(device))
        # run mode
        # run_eval(targetFile, modelPath, str(replica), Path(outDir), device)
        p = ctx.Process(target=run_eval, args=(targetFile, modelPath, str(replica), Path(outDir), device))
        processes.append(p)
        p.start()
        if len(processes) == numParallelProcesser:
            for p in processes:
                p.join()
            processes = []

if len(processes) > 0:
    for p in processes:
        p.join()
    processes = []

print("All models have been running for %s seconds ---" % (time.time() - startTime))

Lauching model: D 0 on device cuda:0
Input file: /data/data/T1019s2/T1019s2.npy
Output dir: /data/data/T1019s2/T1019s2_out/Distogram/0
Distogram model: /data/models/873731
Replica: 0
Device: cuda:0
Lauching model: B 0 on device cuda:1
Input file: /data/data/T1019s2/T1019s2.npy
Output dir: /data/data/T1019s2/T1019s2_out/Background/0
Background model: /data/models/916425
Replica: 0
Device: cuda:1
Lauching model: T 0 on device cuda:0
Input file: /data/data/T1019s2/T1019s2.npy
Output dir: /data/data/T1019s2/T1019s2_out/Torsion/0
Torsion model: /data/models/941521
Replica: 0
Device: cuda:0
Lauching model: D 1 on device cuda:1
Input file: /data/data/T1019s2/T1019s2.npy
Output dir: /data/data/T1019s2/T1019s2_out/Distogram/1
Distogram model: /data/models/873731
Replica: 1
Device: cuda:1
Lauching model: B 1 on device cuda:0
Input file: /data/data/T1019s2/T1019s2.npy
Output dir: /data/data/T1019s2/T1019s2_out/Background/1
Background model: /data/models/916425
Replica: 1
Device: cuda:0
Lauching m

In [ ]:
print("Ensembling all replica outputs & Pasting contact maps")
ensemble(targetFile, Path(outputDir))

In [ ]:
sys.path.append('../alphafoldv1')

import pickle
import tensorflow as tf  

sys.path.append('../alphafoldv1')

from make_constraints import *
from make_torsion_stats import *

TARGET_TORSION = TARGET_DIR + TARGET_NAME + '_out/Torsion/0/' + TARGET_NAME + '.torsion'
TARGET_PDB_TEMP_OUT = TARGET_DIR + TARGET_NAME + '_out/' + TARGET_NAME + '.pickle'

f = tf.io.gfile.GFile(TARGET_TORSION, 'rb')
torsion_probs = pickle.load(f, encoding='latin1')

seq_file = Path(TARGET_SEQ)
target_line, *seq_line = seq_file.read_text().split('\n')
target = seq_file.stem
suffix = seq_file.suffix
sequence = ''.join(seq_line)


PhiPsiList = []

for i in range(torsion_probs.shape[0]):
    print("Calculating torsion stats for residue %d" %(i+1))
    phipsi_stat = CalcTorsionStats(torsion_probs[i])
    print(phipsi_stat)
    PhiPsiList.append(phipsi_stat)

save_torsions_stats(TARGET_PDB_TEMP_OUT,PhiPsiList,sequence)

print("Finish saving file")


In [ ]:
TARGET_DISTENCE = TARGET_DIR + TARGET_NAME + '_out/Distogram/ensemble/' + TARGET_NAME + '.distance'
TARGET_HIST = TARGET_DIR + TARGET_NAME + '_out/' + TARGET_NAME + '_hist'
TARGET_OUT = TARGET_DIR + TARGET_NAME + '_out/' + TARGET_NAME + '_test'

f = tf.io.gfile.GFile(TARGET_DISTENCE, 'rb')

probs = pickle.load(f, encoding='latin1')

potential = CalcPotentialByDFIRE(probs)
pairConstraints = GenerateSplinePotential4Distance(sequence,potential)
WriteSplineConstraints(pairConstraints, savefile=TARGET_OUT, savefolder4histfile=TARGET_HIST)


f = tf.io.gfile.GFile(TARGET_PDB_TEMP_OUT, 'rb')

contact_dict = pickle.load(f, encoding='latin1')

#print(contact_dict.keys())
PhiPsiList = contact_dict['torsion_stats']
sequence = contact_dict['sequence']

#print(PhiPsiList)

PhiPsiConstraints = GeneratePhiPsiPotential(sequence, PhiPsiList)
#print(PhiPsiConstraints)


with open(TARGET_OUT, 'a') as fh:
    fh.write('\n'.join(PhiPsiConstraints) )
    
print("finished generate temp files")

In [ ]:
%%bash
TARGET="T1019s2"
TARGET_DIR="/data/data/T1019s2"

cd /alphafoldv1
python3 fold.py --fasta ${TARGET_DIR}/${TARGET}.seq --constraints ${TARGET_DIR}/${TARGET}_out/${TARGET}_test --out ${TARGET_DIR}/${TARGET}_out/${TARGET}.pdb
python3 make_pdb_image.py --pdb ${TARGET_DIR}/${TARGET}_out/${TARGET}.pdb --sec ${TARGET_DIR}/${TARGET}_out/Torsion/0/${TARGET}.sec -o ${TARGET_DIR}/${TARGET}_out/${TARGET}_pdb.png

In [ ]:
from IPython.display import Image
Image(TARGET_DIR + TARGET_NAME + '_out/' + TARGET_NAME + '_pdb.png')